## Import Packages


In [2]:
from google.colab import drive
drive.mount('/content/drive')


import os
import time
import pprint
import xgboost
import re
import lightgbm as lgb
import numpy as np
import pandas as pd
#import catboost
import matplotlib.pyplot as plt
#from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib import cm


Mounted at /content/drive


In [4]:
bc_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV'
df_bc = pd.read_csv(bc_data)

print(df_bc.info())
df_bc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112785 entries, 0 to 5112784
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   B01         float64
 1   B02         float64
 2   B03         float64
 3   B04         float64
 4   B05         float64
 5   B06         float64
 6   B07         float64
 7   B08         float64
 8   B09         float64
 9   B11         float64
 10  B12         float64
 11  B8A         float64
 12  land_cover  int64  
dtypes: float64(12), int64(1)
memory usage: 507.1 MB
None


,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,land_cover
0,0.0172,0.0200,0.0373,0.0243,0.0643,0.1630,0.1883,0.1995,0.2100,0.1222,0.0642,0.2045,14
1,0.0161,0.0297,0.0476,0.0447,0.0834,0.1770,0.2067,0.2224,0.2265,0.1829,0.1020,0.2302,14
2,0.0140,0.0151,0.0249,0.0241,0.0459,0.0992,0.1188,0.1263,0.2059,0.1082,0.0585,0.1328,14
3,0.0132,0.0267,0.0435,0.0411,0.0794,0.1701,0.1955,0.2137,0.2261,0.1669,0.0943,0.2197,18
4,0.0142,0.0255,0.0422,0.0389,0.0804,0.1700,0.1958,0.2122,0.2349,0.1546,0.0848,0.2189,18
5,0.0149,0.0261,0.0480,0.0372,0.0959,0.2127,0.2442,0.2639,0.2662,0.1509,0.0806,0.2694,18
6,0.0217,0.0289,0.0481,0.0379,0.0863,0.1801,0.2070,0.2180,0.2344,0.1499,0.0834,0.2239,14
7,0.0278,0.0495,0.0721,0.0628,0.1189,0.2124,0.2368,0.2499,0.2588,0.1794,0.1022,0.2572,14
8,0.0153,0.0172,0.0306,0.0266,0.0602,0.1190,0.1369,0.1463,0.1926,0.0895,0.0486,0.1509,2
9,0.0069,0.0066,0.0119,0.0145,0.0233,0.0281,0.0348,0.0316,0.0949,0.0295,0.0158,0.0360,18


In [5]:
on_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV'
df_on = pd.read_csv(on_data)

print(df_on.info())
df_on.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112785 entries, 0 to 5112784
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   B01         float64
 1   B02         float64
 2   B03         float64
 3   B04         float64
 4   B05         float64
 5   B06         float64
 6   B07         float64
 7   B08         float64
 8   B09         float64
 9   B11         float64
 10  B12         float64
 11  B8A         float64
 12  land_cover  int64  
dtypes: float64(12), int64(1)
memory usage: 507.1 MB
None


,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,land_cover
0,0.0172,0.0200,0.0373,0.0243,0.0643,0.1630,0.1883,0.1995,0.2100,0.1222,0.0642,0.2045,14
1,0.0161,0.0297,0.0476,0.0447,0.0834,0.1770,0.2067,0.2224,0.2265,0.1829,0.1020,0.2302,14
2,0.0140,0.0151,0.0249,0.0241,0.0459,0.0992,0.1188,0.1263,0.2059,0.1082,0.0585,0.1328,14
3,0.0132,0.0267,0.0435,0.0411,0.0794,0.1701,0.1955,0.2137,0.2261,0.1669,0.0943,0.2197,18
4,0.0142,0.0255,0.0422,0.0389,0.0804,0.1700,0.1958,0.2122,0.2349,0.1546,0.0848,0.2189,18
5,0.0149,0.0261,0.0480,0.0372,0.0959,0.2127,0.2442,0.2639,0.2662,0.1509,0.0806,0.2694,18
6,0.0217,0.0289,0.0481,0.0379,0.0863,0.1801,0.2070,0.2180,0.2344,0.1499,0.0834,0.2239,14
7,0.0278,0.0495,0.0721,0.0628,0.1189,0.2124,0.2368,0.2499,0.2588,0.1794,0.1022,0.2572,14
8,0.0153,0.0172,0.0306,0.0266,0.0602,0.1190,0.1369,0.1463,0.1926,0.0895,0.0486,0.1509,2
9,0.0069,0.0066,0.0119,0.0145,0.0233,0.0281,0.0348,0.0316,0.0949,0.0295,0.0158,0.0360,18


In [6]:
#df_bc = df_bc[0:SAMPLE_SIZE]
y = df_bc.land_cover
X = df_bc.drop(['land_cover'], axis=1)
columns = X.columns

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
#df_bc = df_bc[0:SAMPLE_SIZE]
y2 = df_on.land_cover
X2 = df_on.drop(['land_cover'], axis=1)
columns2 = X2.columns

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X2= scaler.fit_transform(X2)

In [8]:
print(X.shape)
print(y.shape)
print(X2.shape)
print(y2.shape)

(5112785, 12)
(5112785,)
(5112785, 12)
(5112785,)


In [9]:
X_train=X
y_train=y
X_test=X2
y_test=y2

In [10]:
from sklearn.ensemble import RandomForestClassifier as sklRF 
import multiprocessing as mp 
#skl_rf_params = {‘n_estimators’: 25,‘max_depth’: 13, ‘n_jobs’: mp.cpu_count() }
rf=RandomForestClassifier(n_estimators=50, max_features='sqrt', max_depth=10,min_samples_leaf=2)
                          #max_depth=10,tree_method = "gpu_hist")
#https://developer.nvidia.com/blog/accelerating-random-forests-up-to-45x-using-cuml/

In [11]:
st = time.time()
rf.fit(X_train,y_train)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

Total Elapsed time = 13.7 min


In [12]:
st = time.time()
y_rfpred=rf.predict(X_test)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

Total Elapsed time = 0.7 min


In [13]:
#print("Accuracy:", metrics.accuracy_score(y_test, y_bpred))
from sklearn.metrics import balanced_accuracy_score
print('RF Model accuracy score: {0:0.4f}'.format(metrics.accuracy_score(y_test, y_rfpred)*100),'%')
print('RF Model Balanced Accuracy: {0:0.4f}'.format(balanced_accuracy_score(y_test, y_rfpred)*100),'%')

RF Model accuracy score: 67.2416 %
RF Model Balanced Accuracy: 36.3323 %


In [14]:
rf_feature_imp = pd.Series(rf.feature_importances_,index=columns).sort_values(ascending=False)
rf_feature_imp

B04    0.232340
B05    0.177430
B8A    0.109640
B11    0.099003
B02    0.092164
B08    0.074098
B07    0.060937
B12    0.046710
B03    0.034376
B06    0.027453
B09    0.023337
B01    0.022513
dtype: float64

# Adaptive Boosting

In [15]:
Adaboost = AdaBoostClassifier(base_estimator=rf,n_estimators=50,learning_rate=0.1,algorithm = 'SAMME')
#tree_method = "gpu_hist")

In [ ]:
st = time.time()
Adaboost.fit(X_train,y_train)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

In [ ]:
st = time.time()
y_abpred=Adaboost.predict(X_test)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

In [ ]:
#print("Accuracy:", metrics.accuracy_score(y_test, y_bpred))
from sklearn.metrics import balanced_accuracy_score
print('RF Model accuracy score: {0:0.4f}'.format(metrics.accuracy_score(y_test, y_abpred)*100),'%')
print('RF Model Balanced Accuracy: {0:0.4f}'.format(balanced_accuracy_score(y_test, y_abpred)*100),'%')

In [ ]:
Adaboost_feature_imp = pd.Series(Adaboost.feature_importances_,index=columns).sort_values(ascending=False)
Adaboost_feature_imp